In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
from pyspark.sql.functions import collect_list, concat_ws
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

runcycleid = etl_helpers.start_run_cycle("factrequestpaymenttransaction")

os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    today = str(datetime.date.today())

    df_existing = spark.sql("SELECT max(TIMESTAMP(runcycleendat)) as runcycleendat from dimruncycle where packagename = 'factRequestPaymentTransaction' and success = 't'")
    df_existing.show()
    maxcreatedate = df_existing.first().runcycleendat
    print(maxcreatedate)
    maxcreatedate_str = maxcreatedate.strftime("%Y-%m-%d %H:%M:%S")
    # maxcreatedate_str = '2023-09-14T14:53:13.949' # test value

    df_existing = spark.sql(f"SELECT distinct foiministryrequestid, min(foirequest_id) as foirequest_id from foi_mod.foiministryrequests where (created_at > '{maxcreatedate_str}' or DATE(updated_at) > '{maxcreatedate_str}') group by foiministryrequestid")



    # test data
    # data = [
    #     (4491,	4491),
    #     (6545,	6545),
    #     (6616,	6616),
    #     (6619,	6619),
    #     (21158,	21143)
    # ]

    # # Define column names
    # columns = ["foirequest_id", "foiministryrequestid" ]

    # # Create DataFrame
    # df_existing = spark.createDataFrame(data, schema=columns)

    # # Show DataFrame

    # df_existing.show(df_existing.count(), truncate=False)

    # comma_list_df = df_existing.select(concat_ws(",", collect_list("foirequest_id")).alias("comma_list"))
    # comma_list = comma_list_df.collect()[0]["comma_list"]

    

    print(comma_list)

    query = f"""MERGE INTO default.factrequestpaymenttransaction dd
        -- USING dimRequests r -- commented out since we are adding sourceoftruth column to each fact table
        -- ON r.foirequestid = dd.foirequestid
        WHEN MATCHED and dd.foirequestid in ({comma_list}) and sourceoftruth = 'FOIMOD' THEN

        UPDATE 
        SET dd.activeflag = 'N'""";

        
    comma_list_df = df_existing.select(concat_ws(",", collect_list("foiministryrequestid")).alias("comma_list"))
    comma_list = comma_list_df.collect()[0]["comma_list"]

    print(comma_list)

    query = f"""
        select * from

(
        SELECT *
        FROM (
            SELECT 
            created_at, 
            updated_at,
            foirequestid,
            ministryrequestid,
                ROW_NUMBER() OVER (
                    PARTITION BY paymentid
                    ORDER BY version DESC
                ) AS rn
            FROM foi_mod.foirequestpayments
        ) sub
        WHERE rn = 1 and updated_at is not null) sq3 
        left join
        (SELECT *
        FROM (
            SELECT 
            get_json_object(feedata, '$.balanceremaining') as invoicedamount, 
            ministryrequestid,
            --*,
                ROW_NUMBER() OVER (
                    PARTITION BY cf.ministryrequestid 
                    ORDER BY cf.created_at DESC
                ) AS rn
            FROM foi_mod.foirequestcfrfees cf
            join foi_mod.foiministryrequests mr on mr.foiministryrequestid = cf.ministryrequestid and mr.version = cf.ministryrequestversion
            WHERE mr.requeststatuslabel = 'onhold' and mr.foiministryrequestid in ({comma_list})
        ) sub
        WHERE rn = 1) sq on sq3.ministryrequestid = sq.ministryrequestid 
       

        join

        (SELECT *
        FROM (
            SELECT get_json_object(feedata, '$.feewaiveramount') as feewaived, --*,
            get_json_object(feedata, '$.amountpaid') as paidamount, 
            get_json_object(feedata, '$.estimatedtotaldue') as estimatedamount, 
            get_json_object(feedata, '$.amountpaid') - get_json_object(feedata, '$.balanceremaining') as prepaymentamount, 
            get_json_object(feedata, '$.refundamount') as refundamount, 
            get_json_object(feedata, '$.amountpaid') as feepaidamount, 
            get_json_object(feedata, '$.estimatedtotaldue') as costestimated, 
            ministryrequestid,
                ROW_NUMBER() OVER (
                    PARTITION BY ministryrequestid 
                    ORDER BY cf.created_at DESC
                ) AS rn
            FROM foi_mod.foirequestcfrfees cf
            where cfrfeestatusid = 2 --approved
        ) sub
        WHERE rn = 1) sq2 on sq2.ministryrequestid = sq.ministryrequestid  --and sq2.cfrfeeid = sq.cfrfeeid

        join (select * from 
        (select order_id, status, request_id, --*,
        row_number() over (
            partition by request_id
            order by created_on
        ) as rn
        
        from foi_mod.payments ) sub where rn = 1
        
        ) sq4 on sq4.request_id = sq3.foirequestid
        left join (select amountpaid, foirequestid  from foi_mod.foirequestapplicationfees af join
        (select * from (select *,
        row_number() over (
            partition by foirawrequestid
            order by version) as rn
        from foi_mod.foirequests rr ) sub where rn = 1) rr on af.rawrequestid = rr.foirawrequestid) sq5 on sq5.foirequestid = sq3.foirequestid
        where sq3.updated_at is not null
        """

    df = spark.sql(query)
    df.show()
    print(df.count())


    # order of columns here is important!
    df_mapped = df.selectExpr(
        "request_id AS foirequestid",
        f"{runcycleid} as runcycleid",
        "'' AS paymentmodeid",
        "estimatedamount AS estimatedamount",
        "NULL AS incurredamount",
        "'' AS interestamount",
        "order_id AS invoicenumber",
        "invoicedamount AS invoicedamount",
        "paidamount AS paidamount",
        "NULL AS willingamount",
        "amountpaid AS applicationfee",
        "'' AS applicationfeetype",
        "'' AS feewaivegranted",
        "'' AS feewaiverequested",
        "'' AS payallfees",
        "'' AS feewaivedescription",
        "created_at AS invoicedate",
        "feewaived AS feewaived",
        "invoicedamount AS totalinvoicedamount",
        "prepaymentamount AS prepaymentamount",
        "'' AS  applicationfeeflag",
        "refundamount AS refundamount",
        "paidamount AS feepaidamount",
        "status AS paymentstatus",
        "costestimated AS costestimated",
        "updated_at AS paymentreceiveddate",
        "'Y' as activeflag",
        "'FOIMOD' as sourceoftruth",
    )
    df_mapped.show(df_mapped.count(), truncate=False)
    df_mapped.write.format("delta").mode("append").option("mergeSchema", "false").insertInto("factrequestpaymenttransaction")  
    etl_helpers.end_run_cycle(runcycleid, 't', "factrequestpaymenttransaction")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', "factrequestpaymenttransaction", "Credentials not available")
    dbutils.notebook.exit("Error: Something went wrong.")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', "factrequestpaymenttransaction")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', "factrequestpaymenttransaction", f"An error occurred: {e}")
        dbutils.notebook.exit("Error: Something went wrong.")